Este colab fue desarrollado por Arnold Charry Armero.

# Ubicación de Instalaciones

Se desean ubicar instalaciones minimizando costos de transporte y costos fijos de abrir instalaciones, por lo cual se le pide formular un modelo de programación entera que refleje esto.

## Conjuntos

$ i = \textrm{Conjunto de posibles localizaciones de plantas, indexado por} \:  i$

$ j = \textrm{Conjunto de mercados, indexado por} \:  j$

## Parámetros

$ D_{j} = \textrm{Demanda anual desde el mercado } j$

$ K_{i} = \textrm{Capacidad de la planta } i$

$ f_{i} = \textrm{Costo fijo anual de mantener la planta} i \textrm{ abierta}$

$ c_{ij} = \textrm{Costo de producir y transportar una unidad desde la planta } i \textrm{ al mercado } j$


## Variables de decisión

$ x_{ij} = \textrm{Cantidad enviada desde la planta } i \textrm{ al mercado } j$

$ y_{i} = \textrm{1 si la planta } i \textrm{ se abre, 0 lo contrario}$

## Función Objetivo

$$ \text{Min} \: \sum_{i = 1}^{n}\sum_{j = 1}^{m}c_{ij} \: x_{ij} + \sum_{i = i}^{n}f_{i}\: y_{i}$$

## Restricciones

Se debe satisfacer la demanda de cada mercado $j$

$$ \sum_{i = 1}^{n}x_{ij} ≥ D_{j}, \: \forall j $$

Una planta no puede producir más que su capacidad

$$ \sum_{j = 1}^{m}x_{ij} \leq K_{i} y_{i}, \: \forall i $$

Se obtiene una variable binaria y una entera positiva

$$ x_{ij}\geq 0, y_{i} \in  \left \{ 0, 1 \right \}$$

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 103.8 MB/s eta 0:00:00


In [ ]:
# Se importan las librerías
import pandas as pd
from gurobipy import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
basic_info = pd.read_excel('/content/drive/MyDrive/Investigación de Operaciones/Gurobi/IP Dataset.xlsx', sheet_name = 'Basic Information')
basic_info.head()

,City,Market
0,"Spokane, WA",Northwest
1,"Reno, NV",Southwest
2,"Omahe, NE",Midwest
3,"Harrisburg, PA",Southwest
4,"Jacksonville, FL",Northwest


In [ ]:
cities = range(len(basic_info['City']))
markets = range(len(basic_info['Market']))

In [ ]:
city_info = pd.read_excel('/content/drive/MyDrive/Investigación de Operaciones/Gurobi/IP Dataset.xlsx', sheet_name = 'City\'s Information')
city_info.head()

,City,Operating cost,Capacity
0,"Spokane, WA",40000,20000
1,"Reno, NV",30000,20000
2,"Omahe, NE",25000,15000
3,"Harrisburg, PA",40000,25000
4,"Jacksonville, FL",30000,15000


In [ ]:
operating_cost = city_info['Operating cost']
capacities = city_info['Capacity']

In [ ]:
market_info = pd.read_excel('/content/drive/MyDrive/Investigación de Operaciones/Gurobi/IP Dataset.xlsx', sheet_name = 'Market\'s Information')
market_info.head()

,Market,Demand
0,Northwest,8000
1,Southwest,12000
2,Midwest,9000
3,Southwest,14000
4,Northwest,17000


In [ ]:
demands = market_info['Demand']

In [ ]:
shipping_info = pd.read_excel('/content/drive/MyDrive/Investigación de Operaciones/Gurobi/IP Dataset.xlsx', sheet_name = 'Shipping Cost')
shipping_info.head()

,Unnamed: 0,Northwest,Southwest,Midwest,Southwest.1,Northwest.1
0,"Spokane, WA",2.40,3.50,4.80,6.80,5.75
1,"Reno, NV",3.25,2.30,3.40,5.25,6.00
2,"Omahe, NE",4.05,3.25,2.85,4.30,4.75
3,"Harrisburg, PA",5.25,6.05,4.30,3.25,2.75
4,"Jacksonville, FL",6.95,5.85,4.80,2.10,3.50


In [ ]:
shipping_cost = []
for i in shipping_info.index:
  shipping_cost.append(list(shipping_info.loc[i]))

shipping_cost = [[float(str(x).replace(',', '.')) for x in fila[1:]] for fila in shipping_cost]

In [ ]:
# Crear la instancia del modelo
model = Model("UbicaciónInstalaciones")

# Inicialización de las variables de decisión
X = model.addVars(cities, markets, vtype=GRB.INTEGER, lb=0, name="X")
Y = model.addVars(cities, vtype=GRB.BINARY, name="Y")

# Agregar la función objetivo
model.setObjective(quicksum(shipping_cost[i][j] * X[i,j] for i in cities for j in markets)
    + quicksum(operating_cost[i] * Y[i] for i in cities), GRB.MINIMIZE)

# Agregar las restricciones

for j in markets:
    model.addConstr(quicksum(X[i,j] for i in cities) >= demands[j], name=f"Demanda_{j}")

for i in cities:
    model.addConstr(quicksum(X[i,j] for j in markets) <= capacities[i] * Y[i], name=f"Capacidad_{i}")

model.setParam('IntFeasTol', 1e-9)
model.setParam('FeasibilityTol', 1e-9)
model.setParam('OptimalityTol', 1e-9)

# Resolver el problema
model.optimize()

# Imprimir el estado del problema
if model.Status == GRB.OPTIMAL:
    print("Status: Optimal")

# Resultados
for var in model.getVars():
    if abs(var.X) < 1e-6:
        var._xround = 0.0
    else:
        var._xround = var.X

print(f"\nFunción Objetivo = {model.objVal:.2f}")

Set parameter IntFeasTol to value 1e-09
Set parameter FeasibilityTol to value 1e-09
Set parameter OptimalityTol to value 1e-09
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
FeasibilityTol  1e-09
IntFeasTol  1e-09
OptimalityTol  1e-09

Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0xb8dd9365
Variable types: 0 continuous, 30 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+00, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Found heuristic solution: objective 391600.00000
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 0 continuous, 30 integer (5 binary)

Root relaxation: objective 2.528000e+05, 11 iterations, 0.00 seconds (0.00 work units

In [ ]:
for i in cities:
    print(Y[i].VarName, "=", round(abs(Y[i].X), 4))

# Encabezado de la tabla
print("\tMarket1\tMarket2\tMarket3\tMarket4\tMarket5")

for i in cities:
    # Mostrar la ciudad actual
    print("City" + str(i + 1), "\t", end="")

    for j in markets:
        # Redondear y eliminar -0.0
        val = round(abs(X[i, j].X), 4)
        print(f"{val}\t", end="")

    print("")  # salto de línea entre filas

# Imprimir valor de la función objetivo
print(f"z* = {model.objVal:.2f}")

Y[0] = 0.0
Y[1] = 1.0
Y[2] = 0.0
Y[3] = 1.0
Y[4] = 1.0
	Market1	Market2	Market3	Market4	Market5
City1 	0.0	0.0	0.0	0.0	0.0	
City2 	8000.0	12000.0	0.0	0.0	0.0	
City3 	0.0	0.0	0.0	0.0	0.0	
City4 	0.0	0.0	8000.0	0.0	17000.0	
City5 	0.0	0.0	1000.0	14000.0	0.0	
z* = 268950.00
